## Programming Exercise 3: Multi-class Classification and Neural Networks
#### Author - Rishabh Jain

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
from scipy.io import loadmat

### 1&nbsp;&nbsp;&nbsp;&nbsp;Multi-class Classification

##### Problem Statement
For this exercise, we will use **logistic regression** and **neural networks** to recognize handwritten digits (from 0 to 9). We are given a dataset in ex3data.mat that contains 5000 training examples of handwritten digits, where each training example is a 20 X 20 pixel grayscale image of the digit. Each pixel is repsented by a floating point number indicating the grayscale intensity at that location.  

This 20 by 20 pixel grid is "unrolled" into 400-dimensional vector. Each of these training examples become a single row in our dataset. This gives us a 5000 X 400 design matrix X where every row is a training example for a handwritten digit image. The second part of the training set is a 5000-dimnesional vectory y that contains labels for the training set

In [62]:
# Loading Mat file
mat=loadmat('./ex3data1.mat')
m=mat['X'].shape[0]
n=mat['X'].shape[1]

# Loading data from mat to Dataframe
data=pd.DataFrame()
for i in range(n):
    data[f'x{i}']=mat['X'].T[i]
data['y']=mat['y'].reshape(m)

print(f'TRAINING DATASET SHAPE : {data.shape[0]} X {data.shape[1]}')
data.sample(5)

TRAINING DATASET SHAPE : 5000 X 401


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x391,x392,x393,x394,x395,x396,x397,x398,x399,y
3690,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
286,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1642,0.0,0.0,0.0,-0.001039,0.012817,0.113275,-0.006235,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2864,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2876,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
